In [5]:
import geolib.geohash
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
from tqdm.auto import tqdm
from cri98tj.partitioners.Geohash_partitioner import Geohash_partitioner
from cri98tj.normalizers.FirstPoint_normalizer import FirstPoint_normalizer
from cri98tj.selectors.RandomInformationGain_selector import RandomInformationGain_selector
from cri98tj.distancers.Euclidean_distancer import Euclidean_distancer
from sklearn.model_selection import train_test_split
from cri98tj.distancers.Euclidean_distancer import euclideanBestFitting

In [6]:
df = pd.read_csv('../examples/Vehicles Dataset/data/vehicles_preapred.zip').sort_values(by=["tid", "t"])# precision=5, 50 movelet, DTW
df = df[["tid", "class", "c1", "c2", "t"]]
df["c1"] = df.c1/100000
df["c2"] = df.c2/100000
df.head()

,tid,class,c1,c2,t
0,30901,B,42.077160,4.738411,0
1,30901,B,42.077246,4.739088,30
2,30901,B,42.077259,4.739096,60
3,30901,B,42.077369,4.739158,90
4,30901,B,42.077635,4.739343,120


In [7]:
tid_train, tid_test, _, _ = train_test_split(df.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

spatioTemporalCols = ["c1", "c2", "t"]
n_movelets=50
n_jobs = 20
verbose = False
precision = 6

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import accuracy_score
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistance_distancer, \
    InterpolatedRootDistanceBestFitting

normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
distancer = InterpolatedRootDistance_distancer(normalizer=normalizer, spatioTemporalColumns=spatioTemporalCols, n_jobs=n_jobs, verbose=verbose)

res = {}
acc_rig = []
n_mov_rig = []
for i in tqdm(range(15)):
    partitioner = Geohash_partitioner(precision=6, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)
    part = partitioner.fit_transform(df[df.tid.isin(tid_train)].values)
    selector = RandomInformationGain_selector(top_k=int(1.5**i), bestFittingMeasure=InterpolatedRootDistanceBestFitting, movelets_per_class=300, trajectories_for_orderline=100, n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, normalizer=normalizer, verbose=verbose)
    shapelets = selector.fit_transform(part)
    _, dist_np = distancer.fit_transform((df.values, shapelets))

    n_mov_rig.append(dist_np.shape[1])

    clf = RandomForestClassifier(max_depth=3, random_state=3, n_jobs=n_jobs, n_estimators=1000)

    dist_np_df = pd.DataFrame(dist_np)
    X = dist_np_df.drop(columns=[0]).values
    y = dist_np_df[0].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    acc_rig.append(accuracy_score(y_test, y_pred))

    res[str(precision)] = classification_report(y_test, y_pred)

In [ ]:
from cri98tj.selectors.Random_selector import Random_selector
from sklearn.metrics import accuracy_score
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistance_distancer, \
    InterpolatedRootDistanceBestFitting

normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
distancer = InterpolatedRootDistance_distancer(normalizer=normalizer, spatioTemporalColumns=spatioTemporalCols, n_jobs=n_jobs, verbose=verbose)

res = {}
acc_r = []
n_mov_r = []
for i in tqdm(range(15)):
    partitioner = Geohash_partitioner(precision=6, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)
    part = partitioner.fit_transform(df[df.tid.isin(tid_train)].values)
    selector = Random_selector(movelets_per_class=max(1, int((1.5**i)/3)), n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, normalizer=normalizer, verbose=verbose)
    shapelets = selector.fit_transform(part)
    _, dist_np = distancer.fit_transform((df.values, shapelets))

    n_mov_r.append(dist_np.shape[1])

    print(f"{int((1.8**i)/3)} - {dist_np.shape}")

    clf = RandomForestClassifier(max_depth=3, random_state=3, n_jobs=n_jobs, n_estimators=1000)

    dist_np_df = pd.DataFrame(dist_np)
    X = dist_np_df.drop(columns=[0]).values
    y = dist_np_df[0].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    acc_r.append(accuracy_score(y_test, y_pred))

    res[int((1.5**i)/3)] = classification_report(y_test, y_pred)

In [1]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=n_mov_r, y=acc_r, mode='lines', name='Random selector'))
fig.add_trace(go.Scatter(x=n_mov_rig, y=acc_rig, mode='lines', name='Random info gain selector'))

fig.show()

NameError: name 'n_mov_r' is not defined